## Bibliothèques

In [3]:
import sys
import os
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

## Loading Files

### Set up Path

In [4]:
DIR_PATH = os.getcwd()
DATA_PATH = os.path.join(DIR_PATH + os.sep, "data")
WRLD_PATH = os.path.join(DATA_PATH + os.sep, "World")
BRL_PATH =  os.path.join(WRLD_PATH + os.sep, "Berlin")

### Load *.csv

In [ ]:
listing = pd.read_csv(BRL_PATH + "listing.csv", low_memory=False)
review = pd.read_csv(BRL_PATH + "review.csv", low_memor)